<a href="https://colab.research.google.com/github/harshjoshi23/ANN/blob/main/Annimproved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# predictiong a binary output classificatiion prob

import numpy as np
import matplotlib as plt
import pandas as pd
import seaborn as sns 
plt.rcParams['figure.figsize'] = (15, 5)
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

from sklearn.utils import class_weight
from sklearn.feature_selection import SelectKBest, chi2, RFECV, SelectFromModel
from sklearn.compose import ColumnTransformer, make_column_transformer

from keras.models import load_model
from keras.layers import Dense
from keras.models import load_model




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Reading and Analysis 

In [3]:
# reading the DF 
df = pd.read_csv('/content/drive/MyDrive/Collabdf/Churn_Modelling.csv')

In [4]:
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


No need of null imputation and no date coloum present

In [6]:
dfcopy = df.copy()

In [7]:
df['Exited'].unique()

array([1, 0])

### Feature Engineering and selection

In [8]:
dfcopy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Not seeing much features right now

In [9]:
f_features = [
            'Geography','Gender', 'Tenure','Age','Tenure','Balance','NumOfProducts','NumOfProducts','HasCrCard','EstimatedSalary','IsActiveMember'
]

In [10]:
X = dfcopy[f_features] # save
y = dfcopy['Exited']

In [11]:
X.head(1)

,Geography,Gender,Tenure,Age,Tenure,Balance,NumOfProducts,NumOfProducts,HasCrCard,EstimatedSalary,IsActiveMember
0,France,Female,2,42,2,0.0,1,1,1,101348.88,1


In [12]:
categorical_features = ['Geography','Gender']


## Encodig Our Data 

In [13]:
labelencoder_X = LabelEncoder()
# X['Geography'] =  labelencoder_X.fit_transform(X['Geography'])
X['Gender'] = labelencoder_X.fit_transform(X['Gender'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
X.shape

(10000, 11)

In [15]:
X.head(1) 

,Geography,Gender,Tenure,Age,Tenure,Balance,NumOfProducts,NumOfProducts,HasCrCard,EstimatedSalary,IsActiveMember
0,France,0,2,42,2,0.0,1,1,1,101348.88,1


In [16]:
# ohe=pd.get_dummies(X['Geography'])
# ohe

In [18]:
from sklearn.preprocessing import LabelBinarizer


In [19]:
X = LabelBinarizer().fit_transform(df.Geography)


In [20]:
# ct = ColumnTransformer([("Geography", OneHotEncoder(), [])], remainder = 'passthrough')
# X = ct.fit_transform(X)

In [21]:
X

array([[1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [22]:
# onehotencoder = OneHotEncoder(categorical_features = ['Geography'])
# X = onehotencoder.fir_transform(X).toarray()


## Train Test Split

In [23]:
X = X[:, 1: ]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=1)



In [25]:
display(X_train.shape )
display(X_test.shape )
display(y_train.shape )
display(y_test.shape )


(7500, 2)

(2500, 2)

(7500,)

(2500,)

In [26]:
from  sklearn.preprocessing import StandardScaler 
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [27]:
import keras
from keras.layers import Dense
from keras.models import Sequential



In [28]:
classifier = Sequential()


## Adding Hidden layers

### Adding the first Hidden Layer

Step 1 Randomly intitialize the weight of node to small no By Dense 
<br>
Step 2 Each feature goes to the neural network 
<br>
Step 3 We will use 

*   Rectifier   hidden layer
*   Sigmoidal - op layer 

Step 4 Compare Predicted Results to actual results. Measure the generated error.

Step 5 Back propogation from right to left, the error rate decides by how much we should update the weights.

Step 6 Repeat 1 to 5 batch or one by one 

Step 7 When the whole training set passes through Ann that makes an epoch. Redo more epochs.




### Using Schostic gradient descent 

In [29]:
# Add 1st hidden layer 
#using add method
# All the hyper parameters are defined inside the dense function
'''

output_dim - no of nodes we want to add in hidden layer
How many nodes to add - depends 
No of nodes in hidden = avg of (input and output layer)
or , 
do hyper parameter tuning 

'''

classifier.add(Dense(6, kernel_initializer='he_uniform', activation='relu'))
# unoform fuction is used to distribute weights uniformly 

''' Activation Functions 
Rectifier - hidden layer
Sigmoidal - op layer '''

# input dim par  no of nodes in input layer, i.e. no of ind variabes 
'''because ann is only initialze so it don't know wich layer is input layer 
but we only specifeid for 1st one'''

"because ann is only initialze so it don't know wich layer is input layer \nbut we only specifeid for 1st one"

### Adding second Hidden layer 

In [30]:
classifier.add(Dense(6, kernel_initializer='he_uniform', activation='relu'))


### Adding The output layer

In [31]:
classifier.add(Dense(1, kernel_initializer='he_uniform', activation='sigmoid'))
#we are using our activation function 
# binary outcome 1 mode 
'''we need prob for outcome so we need sigmoidal not relur 
    sigmoidal fn is eart to prob 
'''

'''if our outcome is not binary so we need to input 3 if we have 3 catagory 
and act fn is softmax and is kind of sigmoidal used for more than 2 catogeries'''



'if our outcome is not binary so we need to input 3 if we have 3 catagory \nand act fn is softmax and is kind of sigmoidal used for more than 2 catogeries'

## Compile our ann 

Applying stochastic gradient descent in ann

In [32]:
'''
def compile(optimizer='rmsprop', loss=None, metrics=None, loss_weights=None, weighted_metrics=None, run_eagerly=None, steps_per_execution=None, **kwargs)
Configures the model for training.

'''
# in optimizer we add Stochaztic 
# loss  corrospond to loss fn in Adam method 
'''
        specify a loss fn to find optimal weights 
        Same for a logesic regression 
            loss funtion is going to be a log function i.e. the log loss
    '''
# accuracy critarian  
'''
        metrics = ['accuracy']
    '''

'''
Example:

model.compile(optimizer=tf.keras.optimizer.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])


                       '''

'\nExample:\n\nmodel.compile(optimizer=tf.keras.optimizer.Adam(learning_rate=1e-3),\n              loss=tf.keras.losses.BinaryCrossentropy(),\n              metrics=[tf.keras.metrics.BinaryAccuracy(),\n                       tf.keras.metrics.FalseNegatives()])\n\n\n                       '

In [33]:
'''

adam   BinaryCrossentropy  catagoricalCrossentropy(for more tan 2)

'''



'\n\nadam   BinaryCrossentropy  catagoricalCrossentropy(for more tan 2)\n\n'

In [34]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Choosing No of Ephocs 
No of times we want to train our ann on the whole training set 
and 
how it's accuracy is increased

In [ ]:
# Fit ANN to training set using fit method 
# batch size = dividing into baches 
# epochs = No of epoches 
classifier.fit(X_train,y_train,batch_size= 10 , epochs= 100 )

Epoch 1/100
750/750 [==============================] - 2s 1ms/step - loss: 0.5357 - accuracy: 0.7707
Epoch 2/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4915 - accuracy: 0.7977
Epoch 3/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4914 - accuracy: 0.7977
Epoch 4/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4915 - accuracy: 0.7977
Epoch 5/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4910 - accuracy: 0.7977
Epoch 6/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4911 - accuracy: 0.7977
Epoch 7/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4910 - accuracy: 0.7977
Epoch 8/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4912 - accuracy: 0.7977
Epoch 9/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4912 - accuracy: 0.7977
Epoch 10/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4908 - accura

## Making prediction on test set 

In [ ]:
y_pred = classifier.predict(X_test)



In [ ]:
from sklearn.metrics import classification_report


In [ ]:
# # model_lgb.fit(x_train, y_train, eval_set = [(x_test, y_test)], verbose = 0)
# # display(classifier.plot_importance(classifier))
# ### Classification Report - Train
# print(classification_report(y_train, classifier.predict(X_trai n)))

# ### Classification Report - Test/Validation
# print(classification_report(y_test, y_pred))

In [ ]:
y_pred

In [ ]:
# cm = confusion_matrix(y_test, y_pred, output_dict=True)
# print("\n***** Test *****")
# print(cm)

In [ ]:
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(y_test, y_pred)
# cm

In [ ]:
y_test

## Implementing Kfold

In [ ]:
# k fold is of sikitlearn so we can combine then by keras wrapper

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# implementing K fold
#1st arg is build fn that return the classifier that we build 
def classbuild():
    classifier = Sequential()
    classifier.add(Dense(6, kernel_initializer='he_uniform', activation='relu'))
    classifier.add(Dense(6, kernel_initializer='he_uniform', activation='relu'))
    classifier.add(Dense(1, kernel_initializer='he_uniform', activation='sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
#fn builds and return the classifier 

#creating a global classifier variable 
#builds throuhg Kfold diff from other in train part 
classifier = KerasClassifier( build_fn = classbuild, batch_size= 10 , epochs= 100  ) 
# 10 accuracy by k fold 
#cv is no of train test folds
#n_jobs - ann is going to be trained 10 times so we use it to make comp faster and use all cpus 
accuracy = cross_val_score(estimator = classifier, X = X_train, y = y_train,cv = 10, n_jobs = -1)

# check if where we are in a variance status 

In [ ]:
# This is the relevant accuracy result 
accuracy

In [ ]:
meanaccu = accuracy.mean()
meanvar = accuracy.var()
std = accuracy.std()
display(std)
display(meanaccu)
display(meanvar)

We can see that our mean accuracyis near 86% or 85.7% to be precise
And mean variance is nearly 0
 

## Tweaking weights and more parameters 